# Rabi Experiments - Comprehensive Examples

This notebook provides comprehensive examples of Rabi oscillation experiments in LeeQ.

## Contents
- Basic Rabi oscillations
- Amplitude vs frequency Rabi experiments
- Multi-level Rabi experiments
- Advanced analysis techniques
- Troubleshooting common issues

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit
import warnings
warnings.filterwarnings('ignore')

# Core LeeQ imports
from leeq.chronicle import Chronicle, log_and_record
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
from leeq.experiments.experiments import ExperimentManager
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon

# Import Rabi experiments using EPII v0.2.0 canonical names
from leeq.experiments.builtin.basic.calibrations import (
    RabiAmplitudeCalibration,
    RabiFrequencyCalibration,
    MeasurementStatistics
)
from leeq.experiments.builtin.basic.characterizations import (
    QubitSpectroscopyFrequency,
    QubitSpectroscopyAmplitudeFrequency
)

# Start Chronicle logging
Chronicle().start_log()
log_and_record("rabi_examples_start", {
    "notebook": "rabi_experiments", 
    "focus": "comprehensive_rabi_examples"
})

print("Rabi Experiments - LeeQ Example Notebook")
print("Using EPII v0.2.0 canonical experiment names")
print("Chronicle logging started")

# Setup simulation environment
manager = ExperimentManager()
manager.clear_setups()

# Create virtual transmon optimized for Rabi demonstrations
rabi_qubit = VirtualTransmon(
    name="RabiDemoQubit",
    qubit_frequency=5100.0,  # MHz
    anharmonicity=-205,
    t1=90,  # Excellent T1 for clear oscillations
    t2=50,  # Good T2* 
    readout_frequency=9800.0,
    quiescent_state_distribution=np.asarray([0.92, 0.06, 0.015, 0.005])
)

# Setup high-level simulation
setup = HighLevelSimulationSetup(
    name='RabiDemoSetup',
    virtual_qubits={1: rabi_qubit}
)
manager.register_setup(setup)

print(f"Rabi demonstration system configured:")
print(f"  Qubit: {rabi_qubit.name}")  
print(f"  Frequency: {rabi_qubit.qubit_frequency} MHz")
print(f"  T1: {rabi_qubit.t1} μs (excellent for clear oscillations)")
print(f"  Anharmonicity: {rabi_qubit.anharmonicity} MHz")

## Basic Rabi Oscillation

A Rabi oscillation experiment reveals the relationship between drive amplitude and qubit state excitation. This is fundamental for pulse calibration.

### Theory
For a resonant drive, the probability of measuring the excited state is:
$$P_1(\Omega_R t) = \sin^2\left(\frac{\Omega_R t}{2}\right)$$

where $\Omega_R$ is the Rabi frequency proportional to the drive amplitude.

### Key Features
- **π-pulse**: First maximum gives the amplitude for complete state flip (|0⟩ → |1⟩)  
- **π/2-pulse**: First π/4 point creates equal superposition
- **Rabi frequency**: Determines how fast the qubit oscillates between states

In [ ]:
# Create and run basic Rabi amplitude calibration experiment
print("=" * 60)
print("BASIC RABI OSCILLATION EXPERIMENT")
print("=" * 60)

# Configure Rabi experiment with optimal parameters for clear oscillations
basic_rabi = RabiAmplitudeCalibration(
    name="BasicRabiOscillation",
    qubit=1,
    drive_frequency=rabi_qubit.qubit_frequency,
    amplitude_start=0.0,
    amplitude_stop=1.2,  # Go beyond first oscillation
    amplitude_points=61,  # High resolution
    pulse_width=0.05,     # Standard 50ns pulse
    repeated_measurement_count=1500  # High statistics for clean data
)

print(f"Rabi experiment configured:")
print(f"  Drive frequency: {basic_rabi.drive_frequency} MHz")
print(f"  Amplitude range: {basic_rabi.amplitude_start} - {basic_rabi.amplitude_stop}")
print(f"  Number of points: {basic_rabi.amplitude_points}")
print(f"  Pulse width: {basic_rabi.pulse_width} μs")
print(f"  Shots per point: {basic_rabi.repeated_measurement_count}")

# Run the experiment (using constructor-only pattern)
print("\nRunning Rabi experiment...")
rabi_results = basic_rabi.run()

# Extract data
amplitudes = rabi_results['sweep_values']
excited_probabilities = rabi_results['measurement_probabilities']

print(f"\nRabi experiment completed!")
print(f"  {len(amplitudes)} data points acquired")
print(f"  Amplitude range: {amplitudes[0]:.3f} to {amplitudes[-1]:.3f}")
print(f"  Excited probability range: {min(excited_probabilities):.3f} to {max(excited_probabilities):.3f}")

# Find π-pulse amplitude (first maximum)
max_indices = []
for i in range(1, len(excited_probabilities)-1):
    if (excited_probabilities[i] > excited_probabilities[i-1] and 
        excited_probabilities[i] > excited_probabilities[i+1]):
        max_indices.append(i)

if max_indices:
    pi_pulse_idx = max_indices[0]  # First maximum
    pi_pulse_amplitude = amplitudes[pi_pulse_idx]
    pi_pulse_probability = excited_probabilities[pi_pulse_idx]
    
    print(f"\nπ-pulse calibration:")
    print(f"  Amplitude: {pi_pulse_amplitude:.4f}")
    print(f"  Excited probability: {pi_pulse_probability:.3f}")
    print(f"  π/2 amplitude: {pi_pulse_amplitude/2:.4f}")
    
    # Find approximate Rabi frequency by measuring oscillation period
    if len(max_indices) >= 2:
        amplitude_period = amplitudes[max_indices[1]] - amplitudes[max_indices[0]]
        estimated_rabi_freq = 1.0 / (amplitude_period * basic_rabi.pulse_width)  # MHz
        print(f"  Estimated Rabi frequency: {estimated_rabi_freq:.1f} MHz (at π-pulse amplitude)")
else:
    pi_pulse_amplitude = amplitudes[np.argmax(excited_probabilities)]
    print(f"\nπ-pulse amplitude (peak): {pi_pulse_amplitude:.4f}")

# Log results to Chronicle
log_and_record("basic_rabi_results", {
    "pi_pulse_amplitude": pi_pulse_amplitude,
    "max_excited_probability": max(excited_probabilities),
    "amplitudes": amplitudes.tolist(),
    "probabilities": excited_probabilities.tolist(),
    "drive_frequency": basic_rabi.drive_frequency,
    "pulse_width": basic_rabi.pulse_width
})

## Advanced Rabi Techniques

This section demonstrates advanced Rabi experiments and analysis techniques including:

1. **Rabi vs Frequency**: 2D spectroscopy to find optimal drive frequency
2. **Power-dependent Rabi**: Understanding amplitude scaling
3. **Damped Rabi oscillations**: Accounting for T1 and T2* effects
4. **Multi-level effects**: Leakage to |2⟩ state analysis
5. **Rabi fitting and parameter extraction**

In [ ]:
# Comprehensive Rabi visualization and analysis
print("\n" + "=" * 60)
print("ADVANCED RABI ANALYSIS AND VISUALIZATION") 
print("=" * 60)

# Create comprehensive Rabi visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Rabi Oscillations',
        'Rabi Frequency vs Amplitude',
        'Rabi Oscillations (Rabi Angle)',
        'Power Dependence Analysis'
    ),
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'scatter'}, {'type': 'scatter'}]]
)

# Plot 1: Basic Rabi oscillations
fig.add_trace(
    go.Scatter(
        x=amplitudes,
        y=excited_probabilities,
        mode='lines+markers',
        name='Rabi Data',
        line=dict(color='blue', width=2),
        marker=dict(size=4)
    ),
    row=1, col=1
)

# Mark π-pulse
if 'pi_pulse_amplitude' in locals():
    fig.add_vline(
        x=pi_pulse_amplitude,
        line_dash="dash",
        line_color="red",
        annotation_text="π pulse",
        row=1, col=1
    )
    fig.add_vline(
        x=pi_pulse_amplitude/2,
        line_dash="dash", 
        line_color="orange",
        annotation_text="π/2 pulse",
        row=1, col=1
    )

# Plot 2: Rabi frequency vs amplitude (linear approximation)
# In the small angle limit: Ω_R ∝ amplitude
rabi_frequencies = amplitudes / (basic_rabi.pulse_width * 2 * np.pi)  # Rough estimate
fig.add_trace(
    go.Scatter(
        x=amplitudes,
        y=rabi_frequencies,
        mode='lines',
        name='Rabi Frequency',
        line=dict(color='green', width=2)
    ),
    row=1, col=2
)

# Plot 3: Rabi angle representation
# Convert amplitude to Rabi angle using calibrated π-pulse
if 'pi_pulse_amplitude' in locals():
    rabi_angles = amplitudes * (np.pi / pi_pulse_amplitude)  # Radians
    fig.add_trace(
        go.Scatter(
            x=rabi_angles,
            y=excited_probabilities,
            mode='lines+markers',
            name='vs Rabi Angle',
            line=dict(color='purple', width=2),
            marker=dict(size=4)
        ),
        row=2, col=1
    )
    
    # Add theoretical sin²(θ/2) curve
    theoretical_angles = np.linspace(0, 3*np.pi, 100)
    theoretical_prob = np.sin(theoretical_angles/2)**2
    fig.add_trace(
        go.Scatter(
            x=theoretical_angles,
            y=theoretical_prob,
            mode='lines',
            name='sin²(θ/2) theory',
            line=dict(color='red', dash='dash', width=2),
            showlegend=True
        ),
        row=2, col=1
    )

# Plot 4: Power dependence analysis
# Show how Rabi frequency scales with amplitude
amplitude_powers = amplitudes**2  # Power ∝ amplitude²
rabi_freq_scaled = rabi_frequencies / np.max(rabi_frequencies)
power_scaled = amplitude_powers / np.max(amplitude_powers)

fig.add_trace(
    go.Scatter(
        x=power_scaled,
        y=rabi_freq_scaled,
        mode='markers',
        name='Power Scaling',
        marker=dict(color='red', size=6)
    ),
    row=2, col=2
)

# Add linear fit line
fig.add_trace(
    go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode='lines',
        name='Linear (ideal)',
        line=dict(color='black', dash='dash', width=2)
    ),
    row=2, col=2
)

# Update layout
fig.update_xaxes(title_text="Amplitude", row=1, col=1)
fig.update_yaxes(title_text="Excited Probability", row=1, col=1)

fig.update_xaxes(title_text="Amplitude", row=1, col=2)
fig.update_yaxes(title_text="Rabi Frequency (MHz)", row=1, col=2)

fig.update_xaxes(title_text="Rabi Angle (rad)", row=2, col=1)
fig.update_yaxes(title_text="Excited Probability", row=2, col=1)

fig.update_xaxes(title_text="Normalized Power", row=2, col=2)
fig.update_yaxes(title_text="Normalized Rabi Freq", row=2, col=2)

fig.update_layout(
    height=800,
    title_text="Comprehensive Rabi Analysis",
    showlegend=True
)

fig.show()

print("\n" + "="*50)
print("ADVANCED RABI EXPERIMENTS")
print("="*50)

# Experiment 2: Rabi vs pulse width (time domain)
print("\n1. Pulse Width Dependence:")
pulse_widths = np.linspace(0.02, 0.2, 8)  # 20ns to 200ns
optimal_amplitude = pi_pulse_amplitude if 'pi_pulse_amplitude' in locals() else 0.5

print(f"Testing pulse widths: {pulse_widths[0]:.2f} to {pulse_widths[-1]:.2f} μs")
print(f"Using fixed amplitude: {optimal_amplitude:.4f}")

width_results = []
for width in pulse_widths:
    # For fixed amplitude, longer pulses give more Rabi rotation
    # π condition: Ω_R * t = π
    rabi_angle = optimal_amplitude * width / (pi_pulse_amplitude * basic_rabi.pulse_width) * np.pi
    excited_prob = np.sin(rabi_angle/2)**2
    width_results.append(excited_prob)
    
width_results = np.array(width_results)

print("Pulse width effects:")
for w, prob in zip(pulse_widths, width_results):
    print(f"  {w:.2f} μs: P₁ = {prob:.3f}")

# Experiment 3: Frequency dependence (detuning effects)
print("\n2. Frequency Detuning Effects:")
detunings = np.linspace(-10, 10, 21)  # ±10 MHz around resonance
base_frequency = rabi_qubit.qubit_frequency

detuning_results = []
for detuning in detunings:
    # Off-resonance reduces effective Rabi frequency
    # Ω_eff = Ω_R * Ω_R / √(Ω_R² + Δ²) for small detunings
    delta = detuning  # MHz
    if abs(delta) < 0.1:  # Near resonance
        reduction_factor = 1.0
    else:
        # Simplified model: Gaussian reduction
        reduction_factor = np.exp(-(delta/5)**2)  # 5 MHz characteristic width
    
    effective_amplitude = pi_pulse_amplitude * reduction_factor
    rabi_angle = np.pi * (effective_amplitude / pi_pulse_amplitude)
    excited_prob = np.sin(rabi_angle/2)**2 * reduction_factor  # Additional detuning penalty
    detuning_results.append(excited_prob)
    
detuning_results = np.array(detuning_results)

print("Frequency detuning effects (π-pulse amplitude):")
print(f"  On resonance (0 MHz): P₁ = {detuning_results[len(detunings)//2]:.3f}")
print(f"  ±5 MHz detuning: P₁ = {detuning_results[len(detunings)//2+5]:.3f}, {detuning_results[len(detunings)//2-5]:.3f}")
print(f"  ±10 MHz detuning: P₁ = {detuning_results[-1]:.3f}, {detuning_results[0]:.3f}")

# Experiment 4: Multi-level analysis (leakage to |2⟩)
print("\n3. Multi-level Analysis:")
print("Analyzing leakage to |2⟩ state during Rabi driving...")

# Simulate leakage based on anharmonicity
anharmonicity = rabi_qubit.anharmonicity  # MHz
leakage_rates = []

for amp in amplitudes[::5]:  # Sample every 5th point for efficiency
    # Higher amplitudes cause more leakage
    # Leakage ∝ (drive_amplitude / anharmonicity)²
    if abs(anharmonicity) > 0:
        leakage_fraction = min(0.1, (amp * 1000 / abs(anharmonicity))**2)  # Cap at 10%
    else:
        leakage_fraction = 0
    leakage_rates.append(leakage_fraction * 100)  # Convert to percentage

print(f"Estimated leakage to |2⟩ (for anharmonicity = {anharmonicity} MHz):")
sample_amps = amplitudes[::5]
for amp, leakage in zip(sample_amps, leakage_rates):
    if amp <= 0.2:  # Show first few points
        print(f"  Amplitude {amp:.3f}: {leakage:.2f}% leakage")

print(f"  Maximum leakage at {sample_amps[-1]:.3f}: {leakage_rates[-1]:.2f}%")

# Create leakage visualization
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=sample_amps,
    y=leakage_rates,
    mode='lines+markers',
    name='Estimated Leakage',
    line=dict(color='red', width=2),
    marker=dict(size=6)
))

fig2.add_hline(
    y=1.0,  # 1% leakage threshold
    line_dash="dash",
    line_color="orange",
    annotation_text="1% threshold"
)

fig2.update_layout(
    title='Multi-level Leakage Analysis',
    xaxis_title='Drive Amplitude',
    yaxis_title='Leakage to |2⟩ (%)',
    width=600,
    height=400
)

fig2.show()

# Summary of advanced analysis
print("\n" + "="*60)
print("RABI ANALYSIS SUMMARY")
print("="*60)
print(f"π-pulse amplitude: {pi_pulse_amplitude:.4f}")
print(f"π/2-pulse amplitude: {pi_pulse_amplitude/2:.4f}")
print(f"Maximum excited probability: {max(excited_probabilities):.3f}")
print(f"Rabi contrast: {max(excited_probabilities) - min(excited_probabilities):.3f}")
print(f"Optimal pulse width: {basic_rabi.pulse_width} μs")
print(f"Drive frequency: {basic_rabi.drive_frequency} MHz")
print(f"Estimated max leakage: {max(leakage_rates):.2f}%")

# Best practices recommendations
print("\n" + "="*40)
print("BEST PRACTICES & RECOMMENDATIONS")
print("="*40)
print("✓ Use π-pulse amplitude for X gates")  
print("✓ Use π/2-pulse amplitude for superposition states")
print("✓ Monitor leakage - keep < 1% for high-fidelity gates")
print("✓ Recalibrate if Rabi contrast drops below 0.8")
print("✓ Consider DRAG pulses if leakage is significant")
print("✓ Track Rabi parameters over time for drift monitoring")

# Log comprehensive results
log_and_record("advanced_rabi_analysis", {
    "pi_pulse_amplitude": pi_pulse_amplitude,
    "rabi_contrast": max(excited_probabilities) - min(excited_probabilities),
    "pulse_width_dependence": {
        "widths_us": pulse_widths.tolist(),
        "excited_probabilities": width_results.tolist()
    },
    "frequency_detuning": {
        "detunings_mhz": detunings.tolist(), 
        "excited_probabilities": detuning_results.tolist()
    },
    "leakage_analysis": {
        "amplitudes": sample_amps.tolist(),
        "leakage_percentages": leakage_rates
    },
    "recommendations": [
        "Use calibrated pi and pi/2 amplitudes",
        "Monitor leakage levels",
        "Track parameters for drift"
    ]
})

print("\nAdvanced Rabi analysis completed and logged to Chronicle!")